In [1]:
import requests as rq
from bs4 import BeautifulSoup as bs

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

In [3]:
import numpy as np
import pandas as pd

import os

### Read NSE 500 companies

In [4]:
input_data_folder = '../data/'

In [5]:
nse_500 = pd.read_csv(os.path.join(input_data_folder, 'ind_nifty500list.csv'))
nse_500.head()
nse_500.shape

,Company Name,Industry,Symbol,Series,ISIN Code
0,3M India Ltd.,SERVICES,3MINDIA,EQ,INE470A01017
1,ACC Ltd.,CEMENT & CEMENT PRODUCTS,ACC,EQ,INE012A01025
2,AIA Engineering Ltd.,INDUSTRIAL MANUFACTURING,AIAENG,EQ,INE212H01026
3,APL Apollo Tubes Ltd.,METALS,APLAPOLLO,EQ,INE702C01019
4,AU Small Finance Bank Ltd.,FINANCIAL SERVICES,AUBANK,EQ,INE949L01017


(501, 5)

In [14]:
# num_companies = 10
num_companies = nse_500.Symbol.nunique()

input_companies_symbol = nse_500['Symbol'].values[:num_companies]
input_companies_symbol[:10]
len(input_companies_symbol)

array(['3MINDIA', 'ACC', 'AIAENG', 'APLAPOLLO', 'AUBANK', 'AAVAS',
       'ABBOTINDIA', 'ADANIGAS', 'ADANIGREEN', 'ADANIPORTS'], dtype=object)

501

### Scrap the data

In [15]:
final_basic_stats_list = []
for company_symbol in input_companies_symbol:
#     url = f'https://www.screener.in/company/{company}/consolidated'
    # look at stand-alone data
    url = f'https://www.screener.in/company/{company_symbol}'
#     print(url)
    response = rq.get(url)
#     print(response.status_code)
    soup = bs(response.text, "html.parser")  # parse the html page
    basic_features_soup = soup.find_all(class_='row-full-width')
#     print(len(basic_features_soup))
    basic_features_list = basic_features_soup[0].find_all(class_='four columns')    
    basic_stats = [f.get_text() for f in basic_features_list]

    basic_stats = [f.lower().strip().replace('\n', '').replace('  ', '').replace(' ', '_') for f in basic_stats]
#     basic_stats
    
    company_stats_dict = {}
    company_stats_dict['symbol'] = company_symbol
    for f in basic_stats:
        s = f.split(":")
        if len(s)==2:
            company_stats_dict[s[0]] = s[1]
#     print(basic_stats_dict)
    final_basic_stats_list.append(list(company_stats_dict.values()))

final_basic_stats_list[:5]

[['3MINDIA',
  '22,361cr.',
  '19,841',
  '1,650',
  '72.54',
  '0.00%',
  '29.16%',
  '19.02%',
  '9.49%',
  '10.00'],
 ['ACC',
  '26,949cr.',
  '1,435',
  '584.47',
  '15.62',
  '0.98%',
  '15.86%',
  '15.14%',
  '8.13%',
  '10.00'],
 ['AIAENG',
  '15,810cr.',
  '1,677',
  '336.80',
  '40.26',
  '0.54%',
  '17.45%',
  '12.34%',
  '15.96%',
  '2.00'],
 ['APLAPOLLO',
  '3,189cr.',
  '1,315',
  '415.57',
  '26.59',
  '1.05%',
  '16.31%',
  '13.15%',
  '25.78%',
  '10.00'],
 ['AUBANK',
  '19,159cr.',
  '652.95',
  '68.15',
  '38.68',
  '0.11%',
  '8.97%',
  '14.70%',
  '41.16%',
  '10.00']]

In [16]:
company_stats_dict.keys()

dict_keys(['symbol', 'market_cap', 'current_price', 'book_value', 'stock_p/e', 'dividend_yield', 'roce', 'roe', 'sales_growth_(3yrs)', 'face_value'])

In [17]:
company_stats_df = pd.DataFrame(final_basic_stats_list, 
                              columns=company_stats_dict.keys())
company_stats_df.head()
company_stats_df.shape

,symbol,market_cap,current_price,book_value,stock_p/e,dividend_yield,roce,roe,sales_growth_(3yrs),face_value
0,3MINDIA,"22,361cr.","19,841","1,650",72.54,0.00%,29.16%,19.02%,9.49%,10.00
1,ACC,"26,949cr.","1,435",584.47,15.62,0.98%,15.86%,15.14%,8.13%,10.00
2,AIAENG,"15,810cr.","1,677",336.80,40.26,0.54%,17.45%,12.34%,15.96%,2.00
3,APLAPOLLO,"3,189cr.","1,315",415.57,26.59,1.05%,16.31%,13.15%,25.78%,10.00
4,AUBANK,"19,159cr.",652.95,68.15,38.68,0.11%,8.97%,14.70%,41.16%,10.00


(501, 10)

In [18]:
change_col_names = {'stock_p/e': 'stock_pe',
                    'sales_growth_(3yrs)': 'sales_growth_3yrs'
                   }
company_stats_df.rename(change_col_names,axis=1, inplace=True)

In [20]:
company_stats_df.sample(5)

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
317,MINDAIND,"8,787cr.",335.10,41.87,62.55,0.33%,15.45%,13.77%,12.65%,2.00
440,TVSMOTOR,"18,705cr.",393.70,70.46,28.09,0.89%,23.67%,21.56%,17.92%,1.00
249,JKPAPER,"2,053cr.",115.20,114.52,4.29,3.04%,24.77%,23.31%,10.14%,10.00
489,VOLTAS,"21,981cr.",664.30,113.13,44.53,0.60%,17.00%,12.32%,9.14%,1.00
288,LTI,"25,741cr.","1,480",271.00,17.05,1.89%,44.24%,33.15%,16.95%,1.00


### Format the data frame

In [21]:
# format values
def format_col_values(x):
    return x.replace('cr.', '').replace(',', '').replace('%', '') 

In [22]:
df = company_stats_df.copy()

In [23]:
df.sample(5)

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
441,TAKE,"1,493cr.",100.95,51.54,,0.99%,2.38%,2.10%,-2.33%,1.00
473,MCDOWELL-N,"44,855cr.",617.30,43.10,55.97,0.00%,20.95%,23.18%,2.88%,2.00
344,OFSS,"26,397cr.","3,075",452.05,21.02,0.00%,51.68%,33.37%,0.49%,5.00
58,BERGEPAINT,"42,278cr.",435.35,24.43,86.48,0.44%,29.93%,19.90%,12.65%,1.00
108,COX&KINGS,56.50cr.,3.20,173.18,0.38,31.25%,8.55%,6.00%,76.06%,5.00


In [24]:
cols_to_process = ['market_cap', 'current_price', 'book_value', 'stock_pe',
       'dividend_yield', 'roce', 'roe', 'sales_growth_3yrs', 'face_value']

for col in cols_to_process:
    df[col] = df[col].apply(lambda x: format_col_values(x))

In [25]:
df.sample(5)

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
128,LALPATHLAB,11171,1340,111.68,55.08,0.45,33.22,22.15,15.03,10.00
64,BHARTIARTL,175282,341.55,191.66,,0.57,0.01,-4.42,-6.30,5.00
225,INDIACEM,2451,79.10,169.08,20.32,1.01,4.79,1.17,9.99,10.00
446,TATAINVEST,4042,798.80,1546,27.62,2.50,2.01,1.85,-10.31,10.00
473,MCDOWELL-N,44855,617.30,43.10,55.97,0.00,20.95,23.18,2.88,2.00


In [26]:
# replace '', '%', '' values as np.NaN
def format_df(df):
    df = df.replace('cr.', np.NaN).replace('%', np.NaN).replace('', np.NaN)
    return df

In [27]:
df = format_df(df)

In [28]:
df.sample(5)

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
150,FORTIS,10422,138.05,110.78,15.92,0.00,5.12,1.98,2.39,10.00
158,GMRINFRA,9899,16.40,19.39,NaN,0.00,1.30,-3.85,-14.74,1.00
247,JKCEMENT,7979,1033,374.39,18.58,0.97,14.35,13.00,6.49,10.00
328,NIITTECH,8642,1385,272.98,31.42,0.00,23.48,18.88,10.44,10.00
474,VGUARD,9663,226.05,21.05,52.64,0.35,25.98,20.07,11.63,1.00


In [29]:
df.isna().sum()

symbol                0
market_cap            0
current_price         0
book_value            1
stock_pe             42
dividend_yield        1
roce                  1
roe                   2
sales_growth_3yrs    12
face_value            0
dtype: int64

In [30]:
df.shape

(501, 10)

### Why values are missing from scraped data

In [31]:
df.sample(5)

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
259,JINDALSAW,2545,79.60,199.08,4.63,2.51,11.90,8.22,17.11,2.00
425,SUNDRMFAST,9601,456.95,90.77,22.76,1.12,27.15,25.03,15.46,1.00
497,ZEEL,24156,251.50,74.25,13.32,1.39,35.52,22.69,17.69,1.00
284,LTTS,16676,1603,234.17,22.12,1.31,42.42,31.82,17.65,2.00
437,TTKPRESTIG,8472,6112,832.29,44.38,0.41,26.80,17.63,9.77,10.00


In [37]:
df.isna().sum()

symbol                0
market_cap            0
current_price         0
book_value            1
stock_pe             42
dividend_yield        1
roce                  1
roe                   0
sales_growth_3yrs    12
face_value            0
dtype: int64

#### Companies having missing sales_growth column

In [32]:
missing_sales_growth = df['sales_growth_3yrs'].isna() == True
df[missing_sales_growth]

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
8,ADANIGREEN,10643,68.05,13.80,NaN,0.00,5.72,-2.45,NaN,10.00
28,ARVINDFASN,2488,424.05,230.04,45.94,0.00,5.97,4.78,NaN,4.00
52,BANDHANBNK,57216,479.55,93.89,26.36,0.63,10.48,18.96,NaN,10.00
118,DALBHARAT,15047,779.80,392.83,161.79,0.26,1.37,1.17,NaN,2.00
146,FINEORG,5176,1688,165.84,40.01,0.41,40.95,28.14,NaN,5.00
212,ISEC,8532,264.85,32.11,18.19,3.55,53.06,52.02,NaN,5.00
236,INDOSTAR,1968,213.30,328.38,7.81,0.94,10.13,9.98,NaN,10.00
271,KPITTECH,2614,95.35,33.69,31.47,0.77,18.50,17.99,NaN,10.00
346,ORIENTELEC,3384,159.50,14.45,44.46,0.63,29.12,24.72,NaN,1.00
435,TCIEXP,2657,692.85,69.66,35.38,0.43,44.23,30.83,NaN,2.00


#### Missing roe

In [36]:
missing_roe = df['roe'].isna() == True
df[missing_roe]

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
430,SUZLON,1303,2.45,-13.20,NaN,0.00,-5.41,NaN,-24.64,2.00
447,TATAMTRDVR,2642,51.95,NaN,NaN,NaN,NaN,NaN,NaN,2.00


In [ ]:
# these companies have no roe; we can replace roe with 0
df.loc[missing_roe, 'roe'] = 0

#### Missing roce

In [38]:
missing_roce = df['roce'].isna() == True
df[missing_roce]

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
447,TATAMTRDVR,2642,51.95,NaN,NaN,NaN,NaN,0,NaN,2.00


#### Missing stock_pe

- Companies that are losing money do not have a P/E ratio

In [46]:
missing_stock_pe = df['stock_pe'].isna() == True
df[missing_stock_pe]

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
8,ADANIGREEN,10643,68.05,13.80,NaN,0.00,5.72,-2.45,NaN,10.00
10,ADANIPOWER,22872,59.30,44.41,NaN,0.00,5.11,-2.40,-42.63,10.00
12,ABCAPITAL,17290,78.50,33.12,NaN,0.00,1.29,0.27,190,10.00
20,ALBK,9900,26.60,24.53,NaN,0.00,0.92,-85.92,-3.70,10.00
25,ANDHRABANK,4924,16.50,44.11,NaN,0.00,4.08,-23.24,2.39,10.00
40,BASF,4268,985.85,326.38,NaN,0.51,1.33,-2.45,8.26,10.00
54,BANKINDIA,19498,59.50,127.20,NaN,0.00,2.98,-15.48,-0.83,10.00
55,MAHABANK,6407,11.00,9.85,NaN,0.00,1.28,-61.01,-5.98,10.00
64,BHARTIARTL,175282,341.55,191.66,NaN,0.57,0.01,-4.42,-6.30,5.00
72,BBTC,7533,1080,36.84,NaN,0.09,2.08,-7.69,0.14,2.00


### Convert to right datatype

In [57]:
df.dtypes

symbol               object
market_cap           object
current_price        object
book_value           object
stock_pe             object
dividend_yield       object
roce                 object
roe                  object
sales_growth_3yrs    object
face_value           object
dtype: object

In [49]:
cols_type = {}
for col in df.columns.values[1:]:
    cols_type[col]='float32'

companies_info = df.astype(cols_type)

In [55]:
companies_info = np.round(companies_info, 2)
companies_info.head(5)
companies_info.dtypes

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
0,3MINDIA,22361.0,19841.000000,1650.000000,72.540001,0.00,29.160000,19.02,9.490000,10.0
1,ACC,26949.0,1435.000000,584.469971,15.620000,0.98,15.860000,15.14,8.130000,10.0
2,AIAENG,15810.0,1677.000000,336.799988,40.259998,0.54,17.450001,12.34,15.960000,2.0
3,APLAPOLLO,3189.0,1315.000000,415.570007,26.590000,1.05,16.309999,13.15,25.780001,10.0
4,AUBANK,19159.0,652.950012,68.150002,38.680000,0.11,8.970000,14.70,41.160000,10.0


symbol                object
market_cap           float32
current_price        float32
book_value           float32
stock_pe             float32
dividend_yield       float32
roce                 float32
roe                  float32
sales_growth_3yrs    float32
face_value           float32
dtype: object

In [60]:
def round_to_two(x):
    return np.round(x, 2)

In [61]:
for col in df.columns.values[1:]:
    companies_info[col] = companies_info[col].apply(lambda x: round_to_two(x))


In [62]:
companies_info.head()

,symbol,market_cap,current_price,book_value,stock_pe,dividend_yield,roce,roe,sales_growth_3yrs,face_value
0,3MINDIA,22361.0,19841.00,1650.00,72.54,0.00,29.16,19.02,9.49,10.0
1,ACC,26949.0,1435.00,584.47,15.62,0.98,15.86,15.14,8.13,10.0
2,AIAENG,15810.0,1677.00,336.80,40.26,0.54,17.45,12.34,15.96,2.0
3,APLAPOLLO,3189.0,1315.00,415.57,26.59,1.05,16.31,13.15,25.78,10.0
4,AUBANK,19159.0,652.95,68.15,38.68,0.11,8.97,14.70,41.16,10.0


### Write to .csv

In [63]:
! pwd

/Users/manishb-imac/personal-projects/clustering-nse-500/notebooks


In [64]:
companies_info.to_csv('../output_data/nse_500_company_info.csv', 
                        index=False)